In [ ]:
import math
import random
import pygame
import tkinter as tk
from tkinter import messagebox

class cube(object):
    rows = 20
    w = 500
    DIS = w/rows
    def __init__(self,start,dirnx=1,dirny=0,color=(255,0,0)):
        self.pos = start
        self.dirnx = 1
        self.dirny = 0
        self.color = color

        
    def move(self, dirnx, dirny):
        self.dirnx = dirnx
        self.dirny = dirny
        self.pos = (self.pos[0] + self.dirnx, self.pos[1] + self.dirny)

    def draw(self, surface, eyes=False):
        pygame.draw.rect(surface, self.color, pygame.Rect(self.pos[0] * self.DIS, self.pos[1] * self.DIS, self.DIS,self.DIS))

        if eyes:
            pygame.draw.circle(surface,(255,255,0),(self.pos[0] * self.DIS + self.DIS//2-5, self.pos[1] * self.DIS + 5),3)
            pygame.draw.circle(surface,(255,255,0),(self.pos[0] * self.DIS + self.DIS//2+5, self.pos[1] * self.DIS + 5),3)

        



class snake(object):
    body = []
    turns = {}
    def __init__(self, color, pos):
        self.color = color
        self.head = cube(pos)
        self.body.append(self.head)
        self.dirnx = 0
        self.dirny = 1

    def move(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

            keys = pygame.key.get_pressed()

            for key in keys:
                if keys[pygame.K_LEFT]:
                    self.dirnx = -1
                    self.dirny = 0
                    self.turns[self.head.pos[:]] = [self.dirnx, self.dirny]

                elif keys[pygame.K_RIGHT]:
                    self.dirnx = 1
                    self.dirny = 0
                    self.turns[self.head.pos[:]] = [self.dirnx, self.dirny]

                elif keys[pygame.K_UP]:
                    self.dirnx = 0
                    self.dirny = -1
                    self.turns[self.head.pos[:]] = [self.dirnx, self.dirny]

                elif keys[pygame.K_DOWN]:
                    self.dirnx = 0
                    self.dirny = 1
                    self.turns[self.head.pos[:]] = [self.dirnx, self.dirny]

        for i,c in enumerate(self.body):
                p = c.pos[:]
                if p in self.turns:
                    turn = self.turns[p]
                    c.move(turn[0], turn[1])
                    if i == len(self.body)-1:
                        self.turns.pop(p)


                else:
                    if c.dirnx == 1 and c.dirny == 0 and c.pos[0] > 19: c.pos = (0,c.pos[1])
                    elif c.dirnx == -1 and c.dirny == 0 and c.pos[0] < 0: c.pos = (19,c.pos[1])
                    elif c.dirny == -1 and c.dirnx == 0 and c.pos[1] < 0: c.pos = (c.pos[0],19)
                    elif c.dirny == 1 and c.dirnx == 0 and c.pos[1] > 19: c.pos = (c.pos[0],0)

                    else:
                        c.move(c.dirnx, c.dirny)
        

    def reset(self, pos):
        self.head = cube(pos)
        self.body = []
        self.body.append(self.head)
        self.turns = {}
        self.dirnx = 0
        self.dirny = 1


    def addCube(self):
        tail = self.body[-1]
        
        if tail.dirnx == 1:
            self.body.append(cube((tail.pos[0]-1, tail.pos[1])))

        elif tail.dirnx == -1:
            self.body.append(cube((tail.pos[0]+1, tail.pos[1])))

        elif tail.dirny == -1:
            self.body.append(cube((tail.pos[0], tail.pos[1]+1)))

        elif tail.dirny == 1:
            self.body.append(cube((tail.pos[0], tail.pos[1]-1)))


        self.body[-1].dirnx = tail.dirnx 
        self.body[-1].dirny = tail.dirny
        

    def draw(self, surface):
        for i, c in enumerate(self.body):
            if i ==0:
                c.draw(surface, True)
            else:
                c.draw(surface)


def drawGrid(win):
    for i in range(0,500,25):
            pygame.draw.rect(win, (255,255,255), pygame.Rect(i,0,1,500))

    for i in range(0,500,25):
            pygame.draw.rect(win,(255,255,255), pygame.Rect(0,i,500,1))


        

def redrawWindow(win):
    global rows, width, s, snack
    win.fill((0,0,0))
    s.draw(win)
    snack.draw(win)
    drawGrid(win)
    pygame.display.update()


def randomSnack(rows, item):

    positions = item.body

    while True:
        x = random.randrange(rows)
        y = random.randrange(rows)
        if len(list(filter(lambda z:z.pos == (x,y), positions))) > 0:
            continue
        else:
            break
        
    return (x,y)


def message_box(subject, content):
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    messagebox.showinfo(subject, content)
    try:
        root.destroy()
    except:
        pass


def main():
    global width, rows, s, snack
    width = 500
    rows = 20
    win = pygame.display.set_mode((width, width))
    s = snake((255,0,0), (10,10))
    snack = cube(randomSnack(rows, s), color=(0,255,0))
    flag = True

    clock = pygame.time.Clock()
    
    while flag:
        pygame.time.delay(50)
        clock.tick(10)
        s.move()
        if s.body[0].pos == snack.pos:
            s.addCube()
            snack = cube(randomSnack(rows, s), color=(0,255,0))

        for x in range(len(s.body)):
            if s.body[x].pos in list(map(lambda z:z.pos,s.body[x+1:])):
                print('Score: ', len(s.body))
                message_box('You Lost!', 'Play again...')
                s.reset((10,10))
                break

            
        redrawWindow(win)

main()